# Retrieval-Augmented Generation (RAG)
This is a notebook used for the 'Methods for Fintech and Artificial Intelligence in Finance'. The notebook experiment in different ways of adding valuable information to the answer generated by an LLM (retrieval-augmented generation, few shot learning). This notebook does not consider fine-tuning LLM as this typically comes with significant cost.

# Setting up the environment

https://community.anaconda.cloud/t/how-do-i-use-an-existing-environment-on-a-new-computer/55641

Load environment with conda
* cd ~/affine

* conda env create -f environment.yml

(If fist time, install a kernel)

* python -m ipykernel install --user --name=**kernel-name**


Remove kernel from system
* jupyter kernelspec list
* jupyter kernelspec uninstall **kernel-name**

Example code:

conda env create -n affine-project-env -f environment.yml

conda activate affine-project-env

python -m ipykernel install --user --name=affine-project-kernel


## Additional notes:
Create venv able with kernel:
* conda env export > environment.yml

In [5]:
%pip --version

pip 24.2 from /opt/miniconda3/envs/affine-project-env/lib/python3.12/site-packages/pip (python 3.12)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#first we will use %pip install to test --> when package should be included in the environment use conda install
%pip install langchain

In [23]:
%pip install -qU "langchain-chroma>=0.1.2"

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install langchain

In [20]:
%pip freeze | grep langchain

langchain==0.3.2
langchain-core==0.3.8
langchain-google-genai==2.0.0
langchain-text-splitters==0.3.0
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [82]:
from dotenv import dotenv_values
ENV = dotenv_values(".env")

In [ ]:
# import os
# from google.colab import userdata
# from langchain_openai import ChatOpenAI
# from langchain_community.document_loaders import UnstructuredFileLoader
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.text_splitter import CharacterTextSplitter
# from langchain.vectorstores import Chroma
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain.schema.runnable import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser
# from langchain.prompts import (
#     ChatPromptTemplate,
#     FewShotChatMessagePromptTemplate,
# )
# # from operator import itemgetter


In [9]:
%pip install --upgrade --quiet  langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


In [27]:
%pip install --upgrade --quiet langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [25]:
from langchain_openai import AzureOpenAI

ModuleNotFoundError: No module named 'langchain_openai'

In [32]:
%pip install --upgrade --quiet langchain-unstructured


Note: you may need to restart the kernel to use updated packages.


In [40]:
%pip uninstall "unstructured[all-docs]" -y

E0000 00:00:1728043318.779637    1496 backup_poller.cc:126] Run client channel backup poller: UNKNOWN:pollset_work {created_time:"2024-10-04T14:01:58.779496946+02:00", children:[UNKNOWN:epoll_wait: Bad file descriptor (9) {created_time:"2024-10-04T14:01:58.779389016+02:00"}]}
Found existing installation: unstructured 0.15.13
Uninstalling unstructured-0.15.13:
  Successfully uninstalled unstructured-0.15.13
Note: you may need to restart the kernel to use updated packages.


In [41]:
# %pip install "unstructured[all-docs]"
#CHANGED BUT NOT TESTED YET
%pip install unstructured-client

Note: you may need to restart the kernel to use updated packages.


In [42]:
from langchain_unstructured import UnstructuredLoader
loader = UnstructuredLoader('book-Copy1.txt')
raw_doc = loader.load()

In [58]:
from langchain_text_splitters import CharacterTextSplitter
# segmenting the document into segments

# Load an example document
with open("book-Copy1.txt") as f:
    book = f.read()

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents([book])
print(texts[0])

# text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
# texts = text_splitter.split_documents(raw_doc)

page_content='﻿The Project Gutenberg eBook of A Christmas Carol
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: A Christmas Carol

Author: Charles Dickens

Illustrator: Arthur Rackham

Release date: December 24, 2007 [eBook #24022]

Language: English

Original publication: Philadelphia and New York: J. B. Lippincott Company,, 1915

Credits: Produced by Suzanne Shell, Janet Blenkinship and the Online
        Distributed Proofreading Team at http://www.pgdp.net


*** START OF THE PROJECT GUTENBERG EBOOK A CHRISTMAS CAROL ***'


In [59]:
from langchain_openai import AzureOpenAIEmbeddings
AZURE_OPENAI_ENDPOINT = ENV.get('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_API_KEY = ENV.get('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_API_VERSION = ENV.get('AZURE_OPENAI_API_VERSION')

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-small",
    # dimensions: Optional[int] = None, # Can specify dimensions with new text-embedding-3 models
    azure_endpoint=AZURE_OPENAI_ENDPOINT, #If not provided, will read env variable AZURE_OPENAI_ENDPOINT
    api_key=AZURE_OPENAI_API_KEY, # Can provide an API key directly. If missing read env variable AZURE_OPENAI_API_KEY
    openai_api_version=AZURE_OPENAI_API_VERSION, # If not provided, will read env variable AZURE_OPENAI_API_VERSION
)

In [60]:
# Document Embedding with Chromadb
from langchain_chroma import Chroma
docsearch = Chroma.from_documents(texts, embeddings)

INFO: HTTP Request: POST https://graphraq-affine-us.openai.azure.com//openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"


In [62]:
#should be renamed later
vectorstore = docsearch

In [72]:
def retrieve_relevant_chunks(chroma_vectorstore, query):
  #Chroma class https://python.langchain.com/api_reference/chroma/vectorstores/langchain_chroma.vectorstores.Chroma.html
  # Connection to query with Chroma indexing using a retriever
  retriever = vectorstore.as_retriever(
      search_type="similarity",
      search_kwargs={'k':4}
  )
  
  chunks = retriever.invoke(query)
  return chunks
  # for d in docs:
  #   print('--------------------------------NEW DOCS ----------------------------------------')
  #   print(d.page_content)

# Function to add all docs returned by retriever

# def format_docs(docs):
#   return "\n\n".join(doc.page_content for doc in docs)

# See difference in answer with context and no context

In [75]:
type(first_chunk.page_content)

str

In [76]:
from langchain_google_genai import GoogleGenerativeAI
GEMINI_API_KEY = ENV.get('GEMINI_API_KEY')
# initializing the LLM

query = "Who are the main characters of the book A Christmas Carol"

llm = GoogleGenerativeAI(model="models/gemini-1.5-flash", api_key=GEMINI_API_KEY)
response_without_context = llm.invoke(query)

chunks = retrieve_relevant_chunks(vectorstore, query)
first_chunk = chunks[0]
rag_query = "\n\n".join([first_chunk.page_content, "\n\n", query])
response_with_context = llm.invoke(rag_query)

INFO: HTTP Request: POST https://graphraq-affine-us.openai.azure.com//openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"


In [77]:
response_without_context

"The main characters of Charles Dickens' *A Christmas Carol* are:\n\n* **Ebenezer Scrooge:** The central character, a miserly and bitter old man who despises Christmas and everything it represents. He undergoes a profound transformation throughout the story.\n* **The Ghosts:**\n    * **The Ghost of Christmas Past:** Shows Scrooge his own past and the events that shaped him into the man he is.\n    * **The Ghost of Christmas Present:** Shows Scrooge the present-day lives of those around him, including his clerk, Bob Cratchit, and his nephew, Fred.\n    * **The Ghost of Christmas Yet to Come:** Shows Scrooge a bleak vision of his future if he does not change his ways.\n* **Bob Cratchit:** Scrooge's loyal and kind-hearted clerk who struggles to provide for his family, including his beloved son, Tiny Tim.\n* **Fred:** Scrooge's nephew, a cheerful and optimistic man who invites Scrooge to Christmas dinner despite his uncle's aversion to the holiday.\n* **The Cratchit Family:** Bob's wife, M

In [79]:
first_chunk.page_content

'Credits: Produced by Suzanne Shell, Janet Blenkinship and the Online\n        Distributed Proofreading Team at http://www.pgdp.net\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK A CHRISTMAS CAROL ***\n\n\nProduced by Suzanne Shell, Janet Blenkinship and the Online\nDistributed Proofreading Team at http://www.pgdp.net\n\n  A CHRISTMAS CAROL\n\n  [Illustration: _"How now?" said Scrooge, caustic and cold as ever.\n  "What do you want with me?"_]\n\n\n  A CHRISTMAS CAROL\n\n  [Illustration]\n\n  BY\n\n  CHARLES DICKENS\n\n  [Illustration]\n\n  ILLUSTRATED BY ARTHUR RACKHAM\n\n  [Illustration]\n\n  J. B. LIPPINCOTT COMPANY PHILADELPHIA AND NEW YORK\n\n  FIRST PUBLISHED 1915\n\n  REPRINTED 1923, 1927, 1932, 1933, 1934, 1935, 1947, 1948, 1952, 1958,\n  1962, 1964, 1966, 1967, 1969, 1971, 1972, 1973\n\n  ISBN: 0-397-00033-2\n\n  PRINTED IN GREAT BRITAIN\n\n\n  PREFACE'

In [78]:
response_with_context

'The main characters of "A Christmas Carol" are:\n\n* **Ebenezer Scrooge:** The miserly protagonist who transforms through the story.\n* **The Ghost of Christmas Past:** A spirit who shows Scrooge his past life and the choices that led him to his current state.\n* **The Ghost of Christmas Present:** A spirit who shows Scrooge the present-day happiness and joy of others, contrasting with his own isolation.\n* **The Ghost of Christmas Yet to Come:** A silent, ominous spirit who shows Scrooge a glimpse of his bleak future if he doesn\'t change his ways.\n* **Bob Cratchit:** Scrooge\'s kind and hardworking clerk, who struggles to provide for his family.\n* **Tiny Tim:** Bob Cratchit\'s young, disabled son, who embodies innocence and optimism.\n* **Fred:** Scrooge\'s nephew, who represents generosity and goodwill. \n'

In [83]:
import requests

NEWS_API_KEY = ENV.get('NEWS_API_KEY')
url = 'https://newsapi.org/v2/everything?q=Apple&from=2024-10-03&language=en&sortBy=popularity&apiKey=' + NEWS_API_KEY
params = {'key':'value'}
r = requests.get(url = url) 
response = r.json() 